In [1]:
import numpy as np
import pandas as pd
import talib as ta
import yfinance as yf

In [2]:
df = yf.download("MSFT", start="2021-07-10", end="2022-07-06", interval="1h", progress=False)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-07-12 09:30:00,279.157013,279.769989,276.950012,277.144989,277.144989,4237731
2021-07-12 10:30:00,277.160004,277.970001,276.825012,277.399994,277.399994,1785328
2021-07-12 11:30:00,277.420013,277.449493,276.579987,277.029999,277.029999,1483945
2021-07-12 12:30:00,277.070007,277.459991,277.029999,277.209991,277.209991,969921
2021-07-12 13:30:00,277.220001,277.839996,277.100006,277.679993,277.679993,1087531


In [3]:
df = df[["Open", "High", "Low", "Close", "Volume"]]
df.dropna(inplace=True)

df.head()

C:\Users\anish\AppData\Local\Temp\ipykernel_12436\1188362657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,Open,High,Low,Close,Volume
Datetime,,,,,
2021-07-12 09:30:00,279.157013,279.769989,276.950012,277.144989,4237731
2021-07-12 10:30:00,277.160004,277.970001,276.825012,277.399994,1785328
2021-07-12 11:30:00,277.420013,277.449493,276.579987,277.029999,1483945
2021-07-12 12:30:00,277.070007,277.459991,277.029999,277.209991,969921
2021-07-12 13:30:00,277.220001,277.839996,277.100006,277.679993,1087531


In [4]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, TrailingStrategy

class SmaCross(TrailingStrategy):
    n1 = 40
    n2 = 200

    def init(self):
        super().init()
        self.set_trailing_sl(2.5)

        high = self.data.High
        low = self.data.Low                
        close = self.data.Close
        self.sma1 = self.I(ta.EMA, close, self.n1)
        self.sma2 = self.I(ta.EMA, close, self.n2)
        self.atr = self.I(ta.ATR, high, low, close, 14)
        self.upperChannel = np.roll(close, 1) + 3*self.atr
        self.lowerChannel = np.roll(close, 1) - 3*self.atr

    def next(self):
        super().next()
        for trade in self.trades:
            if self.data.index[-1] - trade.entry_time >= pd.Timedelta("8 days"):
                if trade.is_long:
                    trade.sl = max(trade.sl, self.data.Low[-1])
                else:
                    trade.sl = min(trade.sl, self.data.High[-1])

        if crossover(self.sma1, self.sma2) or crossover(self.data.Close, self.upperChannel):
            self.sl = self.data.Close[-1] - 2.5*self.atr[-1]
            self.buy(size=0.75, sl=self.sl)

        elif crossover(self.sma2, self.sma1) or crossover(self.data.Close, self.lowerChannel):
            self.sl = self.data.Close[-1] + 2.5*self.atr[-1]
            self.sell(size=0.75, sl=self.sl)

e:\Programming\stock-trading\.conda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
bt = Backtest(df, SmaCross,
            cash=100000, trade_on_close=False, exclusive_orders=True)

# stats = bt.optimize(n1=range(10, 50, 5), n2=range(40, 200, 5), maximize="Return [%]", constraint=lambda p: p.n1 < p.n2)

stats = bt.run()

In [5]:
print("OC-Range: ", (df["Close"].iloc[-1]-df["Close"].iloc[0])/df["Close"].iloc[0])
print("HL-Range: ", (np.max(df["Close"])-np.min(df["Close"]))/np.min(df["Close"]))

OC-Range:  -0.0511464697356281
HL-Range:  0.44387488335262104


In [6]:
stats

Start                     2021-07-12 09:30:00
End                       2022-07-05 15:30:00
Duration                    358 days 06:00:00
Exposure Time [%]                   34.218119
Equity Final [$]                116823.414544
Equity Peak [$]                 119894.141671
Return [%]                          16.823415
Buy & Hold Return [%]               -5.114647
Return (Ann.) [%]                    17.11677
Volatility (Ann.) [%]               14.982924
Sharpe Ratio                         1.142419
Sortino Ratio                          2.2714
Calmar Ratio                         2.671732
Max. Drawdown [%]                    -6.40662
Avg. Drawdown [%]                   -1.301889
Max. Drawdown Duration       73 days 02:00:00
Avg. Drawdown Duration       10 days 14:00:00
# Trades                                   30
Win Rate [%]                             60.0
Best Trade [%]                       6.249475
Worst Trade [%]                     -2.605041
Avg. Trade [%]                    